In [1]:
!pip install brian2
from brian2 import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.1 MB/s eta 0:00:00


In [2]:
import numpy as np

In [3]:
def message_computations(node, p1, p2):
  start_scope()

  #------------------------Input-spikes----------------------

  v1 = array(np.random.binomial(size=1001, n=1, p= p1))
  spike_time = v1*linspace(0, 10000, 1001)
  times1 = array(spike_time[spike_time != 0])*ms
  indices1 = array(ones(len(times1)))

  inp1 = SpikeGeneratorGroup(2,indices1, times1)

  v2 = array(np.random.binomial(size=1001, n=1, p= p2))
  spike_time2 = v2*linspace(0, 10000, 1001)
  times2 = array(spike_time2[spike_time2 != 0])*ms
  indices2 = array(ones(len(times2)))

  inp2 = SpikeGeneratorGroup(2,indices2, times2)


  V_th = 1
  V_rest = 0

  eqs = '''
  dv/dt = (V_rest - v)/tau : 1
  tau : second
  '''
  #------------------------Input-neurons----------------------
  G_in = NeuronGroup(2, eqs, threshold='v>1', reset='v = 0', method='exact')
  G_in.tau = 10*ms

  feedforward1 = Synapses(inp1, G_in, on_pre='v += 1.2')
  feedforward1.connect(j='0')

  feedforward2 = Synapses(inp2, G_in, on_pre='v += 1.2')
  feedforward2.connect(j='1')

  spike_mon = SpikeMonitor(G_in)
  M_in = StateMonitor(G_in, 'v', record=True)

  #------------------------AND1-----------------------
  G_and1 = NeuronGroup(1, eqs, threshold='v>1', reset='v = 0', method='exact')
  G_and1.tau = 5*ms

  S1_and1 = Synapses(G_in, G_and1, on_pre='v += .52')
  S1_and1.connect(i='0')

  S2_and1 = Synapses(G_in, G_and1, on_pre='v += .52')
  S2_and1.connect(i='1')
  M_and1 = StateMonitor(G_and1, 'v', record=True)
  spike_mon_and1 = SpikeMonitor(G_and1)

  #-------------------------Or-------------------------
  G_or = NeuronGroup(1, eqs, threshold='v>0.5', reset='v = 0', method='exact')
  G_or.tau = 5*ms

  S1_or = Synapses(G_in, G_or, on_pre='v += 1.01')
  S1_or.connect(i='0')

  S2_or = Synapses(G_in, G_or, on_pre='v += 1.01')
  S2_or.connect(i='1')
  M_or = StateMonitor(G_or, 'v', record=True)
  spike_mon_or = SpikeMonitor(G_or)

  #------------------------AND2-----------------------
  # We use a negative weight here instead of the NOT gate for simplicity.
  G_and2 = NeuronGroup(1, eqs, threshold='v>0.5', reset='v = 0', method='exact')
  G_and2.tau = 5*ms

  S1_and2 = Synapses(G_and1 , G_and2, on_pre='v += -.52')
  S1_and2.connect(i='0')

  S2_and2 = Synapses(G_or, G_and2, on_pre='v += .52')
  S2_and2.connect(i='0')
  M_and2 = StateMonitor(G_and2, 'v', record=True)
  spike_mon_and2 = SpikeMonitor(G_and2)

  #------------------------NOT----------------------
  G_not = NeuronGroup(1, eqs, threshold='v>1', reset='v = 0', method='exact')
  G_not.tau = 5*ms

  spike_time3 = np.arange(0.5, 10000.5, 10.0)
  times3 = array(spike_time3)*ms
  indices3 = array(zeros(len(times3)))
  in_const = SpikeGeneratorGroup(2,indices3, times3)


  S1_not = Synapses(in_const, G_not, on_pre='v += 1.1')
  S1_not.connect(i='0')

  S2_not = Synapses(G_and2 , G_not, on_pre='v += -1.0')
  S2_not.connect(i='0')


  M_not = StateMonitor(G_not, 'v', record=True)
  spike_mon_not = SpikeMonitor(G_not)

  run(10000*ms)
  if node =="XOR":
    return len(spike_mon_and2.i)/1000
  elif node == "equality":
    return len(spike_mon_and1.i)/len(spike_mon_not.i)
  else:
    return "Select XOR or Equality."

In [ ]:
message_computations("equality", 0.9, 0.1)

0.4885057471264368

In [ ]:
message_computations("equality", 0.18, 0.9)

0.6578947368421053

In [ ]:
message_computations("equality", 0.18, 0.1)

0.02199223803363519

In [ ]:
message_computations("XOR", 0.1, 0.1)

0.174

In [ ]:
message_computations("XOR", 0.5, 0.1)

0.526